In [1]:
# %%
import os
import argparse 

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F


import json as json

import torch.optim as optim

import matplotlib.pyplot as plt
plt.style.use('dark_background')

import models as models

import wandb
# from os import Path

import models 
import datasets
import dataset

import numpy as np
import time as time 
import util.misc as misc
# from util.misc import NativeScalerWithGradNormCount as NativeScaler
from util.callbacks import EarlyStop

from util.engine_train import train_one_epoch, evaluate # evaluate_online


wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: adrian-dendorfer (adrian_s_playground). Use `wandb login --relogin` to force relogin


True

In [2]:
def get_args_parser():
    parser = argparse.ArgumentParser("NN training")

    parser.add_argument('--batch_size', default=64, type=int)
    parser.add_argument('--epochs', default=400, type=int)
    parser.add_argument('--acum_iter', default=1, type=int) 

    parser.add_argument('--model', default='shallow_conv_net', type=str, metavar='MODEL',
                        help='Name of model to train')
    
    # Model parameters
    parser.add_argument('--input_channels', type=int, default=1, metavar='N',
                        help='input channels')
    parser.add_argument('--input_electrodes', type=int, default=61, metavar='N',
                        help='input electrodes')
    parser.add_argument('--time_steps', type=int, default=100, metavar='N',
                        help='input length')
    # parser.add_argument('--length_samples', default=200, 
    #                     help='length of samples') 

    # Optimizer parameters
    parser.add_argument('--optimizer', type=str, default="adam_w", 
                        help='optimizer type') 
    parser.add_argument('--lr', type=float, default=1e-3, metavar='LR',
                        help='learning rate') 

    # Callback parameters
    parser.add_argument('--patience', default=-1, type=float,
                        help='Early stopping whether val is worse than train for specified nb of epochs (default: -1, i.e. no early stopping)')
    parser.add_argument('--max_delta', default=0, type=float,
                        help='Early stopping threshold (val has to be worse than (train+delta)) (default: 0)')


    # Dataset parameters
    parser.add_argument('--data_path', 
                        # default='_.pt',
                        default="/vol/aimspace/users/dena/Documents/mae/data/lemon/data_raw_train.pt",
                        type=str,
                        help='train dataset path')

    parser.add_argument('--labels_path', 
                        # default='_.pt', 
                        default="/vol/aimspace/users/dena/Documents/ad_benchmarking/ad_benchmarking/data/labels_bin_train.pt", #labels_raw_train.pt",
                        type=str,
                        help='train labels path')
    parser.add_argument('--val_data_path', 
                        # default='', 
                        default="/vol/aimspace/users/dena/Documents/mae/data/lemon/data_raw_val.pt",
                        type=str,
                        help='validation dataset path')
    parser.add_argument('--val_labels_path', 
                        # default='_.pt', 
                        default="/vol/aimspace/users/dena/Documents/ad_benchmarking/ad_benchmarking/data/labels_bin_val.pt", # "labels_raw_val.pt"
                        type=str,
                        help='validation labels path')
    parser.add_argument('--number_samples', default=1, type=int, # | str, 
                        help='number of samples on which network should train on. "None" means all samples.')
    
    
    # Wandb parameters
    parser.add_argument('--wandb', action='store_true', default=False)
    parser.add_argument('--wandb_project', default='',
                        help='project where to wandb log')
    parser.add_argument('--wandb_id', default='', type=str,
                        help='id of the current run')
    parser.add_argument('--device', default='cuda',
                        help='device to use for training / testing')
    parser.add_argument('--seed', default=0, type=int)

    # Saving Parameters
    parser.add_argument('--output_dir', default='',
                        help='path where to save, empty for no saving')
    
    # parser.add_argument('--mode', type=str, default="train")

    return parser

In [3]:
class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

In [10]:
args = Namespace(batch_size=16,
    epochs=200,
    acum_iter=1,
    model='shallow_conv_net', # 'simple_regressor', #deep_conv_net', #first_shallow_conv_net_regression', #deep_conv_net', #first_shallow_conv_net_regression', #shallow_conv_net',  # deep_conv_net, simple_classifier
    input_channels=1,
    input_electrodes=61,
    time_steps=210,
    optimizer='adam', #'adamw',
    criterion='ce', # 'mse',   
    lr=0.01,
    patience=1000,
    sufficient_accuracy=1, #-np.inf, 
    max_delta=0,

    # Classification
    data_path='/vol/aimspace/users/dena/Documents/mae/data/lemon/data_raw_train.pt',
    # labels_path='/vol/aimspace/users/dena/Documents/ad_benchmarking/ad_benchmarking/data/labels_bin_train.pt',
    labels_path='/vol/aimspace/users/dena/Documents/ad_benchmarking/ad_benchmarking/data/lemon/labels_categorized_train.pt',
    val_data_path='/vol/aimspace/users/dena/Documents/mae/data/lemon/data_raw_val.pt',
    # val_labels_path='/vol/aimspace/users/dena/Documents/ad_benchmarking/ad_benchmarking/data/labels_bin_val.pt',
    val_labels_path='/vol/aimspace/users/dena/Documents/ad_benchmarking/ad_benchmarking/data/lemon/labels_categorized_val.pt', 

    number_samples=32, #16, #64,
    num_workers=4,
    wandb=False,
    wandb_project='',
    wandb_id='',
    device='cpu', #cuda',
    seed=0,
    output_dir='')


# Training set size:  1
# Validation set size:  1

DeepNet braucht 210 time steps, da 100 zu kurz sind

In [11]:
# HERE IS OVERFITTING OPTION! 

# print('job dir: {}'.format(os.path.dirname(os.path.realpath(__file__))))
# print("{}".format(args).replace(', ', ',\n'))

device = torch.device(args.device)

# Fix the seed for reproducibility
seed = args.seed 
torch.manual_seed(seed)
np.random.seed(seed)


# # wandb logging
# if args.wandb == True:
#     config = vars(args)
#     if args.wandb_id:
#         wandb.init(project=args.wandb_project, id=args.wandb_id, config=config)
#     else:
#         wandb.init(project=args.wandb_project, config=config)
wandb.init(project=args.wandb_project, config=vars(args))


OVERFITTING = True

dataset_train = dataset.EEGDataset(data_path=args.data_path, labels_path=args.labels_path, 
                            train=True, number_samples=args.number_samples, length_samples=args.time_steps, scaled=True,
                            overfitting=OVERFITTING, args=args)
dataset_val = dataset.EEGDataset(data_path=args.data_path, labels_path=args.labels_path, 
                            train=True, number_samples=args.number_samples, length_samples=args.time_steps, scaled=True,
                            overfitting=OVERFITTING, args=args)

print("Training set size: ", len(dataset_train))
print("Validation set size: ", len(dataset_val))

sampler_val = torch.utils.data.SequentialSampler(dataset_val)
sampler_train = torch.utils.data.RandomSampler(dataset_train) 

if OVERFITTING: 
    data_loader_train = torch.utils.data.DataLoader(
        dataset_train, 
        shuffle=False, 
        batch_size=args.batch_size,
        num_workers=args.num_workers,
        drop_last=False,
    )

    data_loader_val = torch.utils.data.DataLoader(
        dataset_train, 
        shuffle=False,
        batch_size=args.batch_size,
        num_workers=args.num_workers,
        drop_last=False,
    )

else: 
    data_loader_train = torch.utils.data.DataLoader(
        dataset_train, 
        sampler=sampler_train,
        # shuffle=True, # since sampler
        batch_size=args.batch_size,
        num_workers=args.num_workers,
        drop_last=False,
    )

    data_loader_val = torch.utils.data.DataLoader(
        dataset_val, 
        sampler=sampler_val,
        # shuffle=False,
        batch_size=args.batch_size,
        num_workers=args.num_workers,
        drop_last=False,
    )


model = models.__dict__[args.model](
    n_channels=args.input_electrodes, 
    input_time_length=args.time_steps, 
)

model.to(device)

# eval_criterion = "bce"
if args.criterion == "bce": 
    criterion = torch.nn.BCELoss() # For classification

elif args.criterion == "ce": 
    criterion = torch.nn.CrossEntropyLoss() # For classification

elif args.criterion == "mae": 
    criterion = torch.nn.L1Loss() # For regression 

elif args.criterion == "mse": 
    criterion = torch.nn.MSELoss() # For regression 



if args.optimizer == "sgd":
    optimizer = optim.SGD(model.parameters(),
                            lr=args.lr, momentum=0.9)
elif args.optimizer == "adam":
    optimizer = optim.Adam(model.parameters(),
                            lr=args.lr)
elif args.optimizer == "adamw": 
    optimizer = optim.AdamW(model.parameters(), lr=args.lr, betas=(0.9, 0.95))

else: 
    print("Attention: No optimier chosen.")

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
mean train acc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mean train loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mean val acc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mean val loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,199
mean train acc,0.5
mean train loss,0.02578
mean val acc,0.5
mean val loss,0.02578


Sample indices:
[11, 22, 10, 2, 16, 14, 28, 26, 20, 13, 24, 5, 17, 8, 30, 25, 23, 1, 31, 6, 4, 18, 29, 19, 9, 7, 27, 3, 0, 21, 15, 12]
Sample indices:
[28, 12, 15, 18, 17, 21, 23, 26, 24, 27, 8, 6, 13, 5, 4, 16, 22, 2, 11, 3, 20, 10, 25, 9, 1, 0, 7, 14, 30, 31, 29, 19]
Training set size:  32
Validation set size:  32


In [12]:
# for now: 
# data_loader_train = data_loader_val

In [13]:
# CLASSIFICATION

# # Define callbacks
# # early_stop = EarlyStop(patience=args.patience, max_delta=args.max_delta)

# print(f"Start training for {args.epochs} epochs")

# min_val_metric = np.inf
# counter = 0 

# for epoch in range(args.epochs): 
    
#     mean_loss_epoch_train_bce, mean_loss_epoch_train_L1 = train_one_epoch(model, data_loader_train, optimizer, criterion, device, epoch, args=args) #loss_scaler, criterion
#     print(f"Loss / BCE on {len(dataset_train)} train samples: {mean_loss_epoch_train_bce}")

#     # mean_loss_epoch_val_bce, mean_loss_epoch_val_L1 = evaluate(model, data_loader_val, criterion, device, epoch, args=args) 
#     target, output, mean_loss_epoch_val_bce, mean_loss_epoch_val_L1 = evaluate(model, data_loader_val, criterion, device, epoch, args=args) 
#     print(target, output) 
#     print(f"Loss / BCE on {len(dataset_val)} val samples BCE: {mean_loss_epoch_val_bce}, val samples MAE: {mean_loss_epoch_val_L1}")
#     wandb.log({"mean train BCE loss": mean_loss_epoch_train_bce,
#                "mean train MAE loss": mean_loss_epoch_train_L1, 
#                "mean val BCE loss": mean_loss_epoch_val_bce, 
#                "mean val MAE loss": mean_loss_epoch_val_L1, 
#                "epoch": epoch})
    
#     # Early Stopping
#     print(f"Sufficient accuracy: {args.sufficient_accuracy}.")
#     print(f"patience: {args.patience > -1}.")
#     print(f"stuff: {mean_loss_epoch_train_L1 < args.sufficient_accuracy}.")
#     if args.patience > -1: 
#         if mean_loss_epoch_train_L1 < args.sufficient_accuracy: 
#             break
#         elif mean_loss_epoch_train_L1 < min_val_metric: 
#             min_val_metric = mean_loss_epoch_train_L1
#             counter == 0
#         elif mean_loss_epoch_train_L1 > min_val_metric: 
#             counter += 1
#             if counter > args.patience:
#                 print(f"stopped early at epoch {epoch}.")
#                 break 



In [14]:
for data in enumerate(data_loader_val): 
    break

data[1][1]

# ALSO ACHTUNG; DER FEHLER IST NICHT IN JAHREN, SONDERN IN JAHREN/100

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0100, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0100, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0100, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0100, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0100, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0100, 0.0000, 0.0

In [16]:
# REGRESSION 

# Define callbacks
# early_stop = EarlyStop(patience=args.patience, max_delta=args.max_delta)

print(f"Start training for {args.epochs} epochs")

min_val_metric = np.inf
counter = 0 

for epoch in range(args.epochs): 
    
    train_loss, train_acc = train_one_epoch(model, data_loader_train, optimizer, criterion, device, epoch, scaled=True, args=args) #loss_scaler, criterion
    print(f"Loss train on {len(dataset_train)} train samples: {train_loss}, Acc: {train_acc}")

    # mean_loss_epoch_val_bce, mean_loss_epoch_val_L1 = evaluate(model, data_loader_val, criterion, device, epoch, args=args) 
    val_loss, val_acc = evaluate(model, data_loader_val, criterion, device, args=args) 
    # print(target, output) 
    print(f"Loss on {len(dataset_val)} val samples: {val_loss}, Acc: {val_acc}")
    wandb.log({"mean train loss": train_loss,
               "mean val loss": val_loss, 
               "mean train acc": train_acc, 
               "mean val acc": val_acc, 
               #    "true age": target, 
            #    "guessed age": output, 
               "epoch": epoch})
    
    # Early Stopping
    # print(f"Sufficient accuracy: {args.sufficient_accuracy}.")
    # print(f"patience: {args.patience > -1}.")
    if args.patience > -1: 
        if train_acc > 0.99: #args.sufficient_accuracy: 
            break

        # elif mean_loss_epoch_val_mae < min_val_metric: 
        #     min_val_metric = mean_loss_epoch_val_mae
        #     counter == 0
        # elif mean_loss_epoch_val_mae > min_val_metric: 
        #     counter += 1
        #     if counter > args.patience:
        #         print(f"stopped early at epoch {epoch}.")
        #         break 



Start training for 200 epochs


/u/home/dena/.conda/envs/mae2/lib/python3.11/site-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


tensor([[2.8391e-03, 4.8727e-06, 5.9100e-02, 1.4794e-03, 9.3609e-01, 6.9685e-06,
         2.8057e-06, 1.0004e-07, 2.2595e-05, 1.2560e-06, 5.9063e-05, 2.9469e-07,
         2.2608e-08, 5.3148e-07, 2.7989e-06, 3.5785e-04, 1.9464e-06, 4.2214e-09,
         3.1049e-05, 2.0607e-07],
        [1.3493e-03, 3.0212e-06, 9.7674e-03, 2.4496e-03, 9.8622e-01, 6.5469e-07,
         1.1521e-06, 1.7811e-08, 1.7883e-05, 5.9519e-07, 1.6681e-05, 5.5653e-08,
         9.1576e-09, 2.6808e-07, 3.9938e-07, 1.6216e-04, 7.0950e-07, 1.9255e-09,
         8.4714e-06, 9.4755e-08],
        [1.4085e-03, 2.7837e-06, 1.1735e-02, 2.1769e-03, 9.8445e-01, 6.7679e-07,
         1.3171e-06, 1.8556e-08, 1.8870e-05, 4.8426e-07, 2.1784e-05, 6.6104e-08,
         9.7403e-09, 2.6111e-07, 4.5193e-07, 1.7272e-04, 6.6455e-07, 2.1023e-09,
         8.0412e-06, 1.1027e-07],
        [1.4188e-03, 2.0648e-06, 1.0370e-02, 1.5118e-03, 9.8651e-01, 7.1338e-07,
         1.0726e-06, 1.7115e-08, 1.5854e-05, 4.0356e-07, 2.1421e-05, 6.6069e-08,
       

Notes: 
- SDG works increadibly bad
- first_shallow_conv_net and deep_conv_net have currently hardcoded eeg channels (n_channels = 61) and input time lengths (input_time_length == 100)
- Created ShallowConvNet_Regression for regression, that does not have a final activation function (the one in the paper has a softmax) 
- age is now scaled to be between 0 and approx. 1 (linearly, by dividing by 100 >> scale instead to fit normal-ish distributon?. Done with "self.y = self.y/100")  
- Final layer of DeepConvNet is currently hardcoded (self.linear_classification = nn.Linear(33200, 1)). Change that. 
